# Checking which device the code is running on

In [1]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3296877975321716312
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7810842624
locality {
  bus_id: 1
  links {
  }
}
incarnation: 4539425842461911283
physical_device_desc: "device: 0, name: GeForce RTX 3080, pci bus id: 0000:09:00.0, compute capability: 8.6"
]


# Optional Installs and Imports

In [2]:
# !pip install scipy
# !pip install pandas
# !pip install sklearn

In [3]:
import tensorflow as tf
import pandas as pd
from tensorflow.keras.layers import Input, Conv2D, Flatten, Dense, MaxPool2D, BatchNormalization, GlobalAveragePooling2D, AveragePooling2D, Dropout, Activation
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import numpy as np
from tensorflow.keras.optimizers import Adam

# Utility Functions for Loading Datasets

In [4]:
def read_image(img_filepath):
    directory = 'chimp_faces/'
    img_file = tf.io.read_file(directory + img_filepath)
    
    #First attempt:
    #img = tf.image.decode_image(img, dtype=tf.float32)
    
    #Second attempt:
    img_tensor_uint8  = tf.io.decode_png(img_file)
    img_tensor_float32 = tf.cast(img_tensor_uint8, tf.float32)
    return img_tensor_float32

def augment(img):
    #insert optional augmentations here
    if tf.random.uniform((), minval=0, maxval=1) < 0.1: #Ten percent probability
        img = tf.image.random_brightness(img, max_delta=0.1)
#     if tf.random.uniform((), minval=0, maxval=1) < 0.1: #Ten percent probability
#         img = tf.image.random_contrast(img, lower=0.1, upper=0.2)
    img = tf.image.random_flip_left_right(img)
    return img

def normalize01(img):
    max = 255
    min = 0
    result = tf.math.subtract(img, min)
    norm_image = tf.math.divide(result, (max - min))
    return norm_image

def resize(img, target_h=224, target_w=224):
    # resize with 0-padding
    img = tf.image.resize_with_pad(img,
                                     target_h,
                                     target_w,
                                     method=tf.image.ResizeMethod.BILINEAR,
                                     antialias=False
                                     )
    return img


def onehot_labels(labels):
    #encode labels to integer
    label_encoder = LabelEncoder()
    integer_encoded_labels = label_encoder.fit_transform(labels)
    #encode labels to one_hot
    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded_labels = integer_encoded_labels.reshape(len(integer_encoded_labels), 1)
    onehot_encoded_labels = onehot_encoder.fit_transform(integer_encoded_labels)
    return onehot_encoded_labels

def load_dataset_split(filename, training_set):
    annotations_directory = 'Freytag_dataset_splits/compiled_skf/'
    df = pd.read_csv(annotations_directory + filename)
    filepaths = df['filename'].values
    names = df['name'].values
    ds_names = tf.data.Dataset.from_tensor_slices(onehot_labels(names))

    ds_images = tf.data.Dataset.from_tensor_slices(filepaths)
    
    ds_images = ds_images.map(read_image)    
    
    ds_images = ds_images.map(normalize01)
    ds_images = ds_images.map(resize)
    if training_set==True:
        ds_images = ds_images.map(augment)

    dataset = tf.data.Dataset.zip((ds_images, ds_names))
    dataset = dataset.shuffle(buffer_size=10000)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
    
    return dataset

# Loading with Freytag Dataset Splits

In [5]:
img_height, img_width = (224, 224)
batch_size = 32
train_ds_list = []
test_ds_list = []
#Each split has 86 distinct names

#Loading different training splits
ds_train1 = load_dataset_split('train_1.csv', training_set=True)
ds_train2 = load_dataset_split('train_2.csv', training_set=True)
ds_train3 = load_dataset_split('train_3.csv', training_set=True)
ds_train4 = load_dataset_split('train_4.csv', training_set=True)
ds_train5 = load_dataset_split('train_5.csv', training_set=True)
train_ds_list.append(ds_train1)
train_ds_list.append(ds_train2)
train_ds_list.append(ds_train3)
train_ds_list.append(ds_train4)
train_ds_list.append(ds_train5)


#Loading different test splits
ds_test1 = load_dataset_split('test_1.csv', training_set=False)
ds_test2 = load_dataset_split('test_2.csv', training_set=False)
ds_test3 = load_dataset_split('test_3.csv', training_set=False)
ds_test4 = load_dataset_split('test_4.csv', training_set=False)
ds_test5 = load_dataset_split('test_5.csv', training_set=False)
test_ds_list.append(ds_test1)
test_ds_list.append(ds_test2)
test_ds_list.append(ds_test3)
test_ds_list.append(ds_test4)
test_ds_list.append(ds_test5)

In [6]:
def generate_model(num_classes=86, weightsname='imagenet', trainable=False, layers_unfrozen=0):
    """
    set trainable to True to retrain certain layers. (ResNet50 has 175 layers.)
    
    With trainable at True, layers_unfrozen = 0 (default value) unfreezes ALL layers in ResNet50
    for training.
    
    layers_unfrozen set to any number other than zero denotes the number of layers of ResNet50 to 
    unfreeze starting from the bottom most layer (N minus layers_unfrozen)
    """

    base_model = ResNet50(include_top=False, weights=weightsname)
    input_layer = tf.keras.Input(shape=(img_height, img_width, 3))
    x = base_model(input_layer, training=trainable)
    
    
    if trainable == False:
        for layer in base_model.layers:
            layer.trainable = False
    else: #trainable == True
        if layers_unfrozen!=0:
            #freeze only a set number of layers
            layers_frozen = len(base_model.layers) - layers_unfrozen
            for i in range(layers_frozen):
                base_model.layers[i].trainable = False
        else:
            #leave all layers unfrozen
            pass
        
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=input_layer, outputs=predictions)
    #model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['accuracy'])
    return model

def train(trainable_or_not, unfrozen_layers_count, epoch_count, save_directory, unlabeled_model_name, train_datasets, test_datasets):
    result_list = []
    loss_list = []
    accuracy_list = []
    for i in range(5):
        print("Split " + str(i+1) + ":")
        #Make sure unlabeled_model_name DOESN'T CONTAIN '.h5'
        labeled_model_name = 'model' + str(i+1) + '-' + unlabeled_model_name + '.h5'
        #Make sure save_directory STARTS WITHOUT a forrward slash '/' and ENDS WITH a forward slash '/'
        save_path = save_directory + labeled_model_name
        model = generate_model(trainable=trainable_or_not, layers_unfrozen=unfrozen_layers_count)
        model.fit(train_datasets[i], epochs=epoch_count)
        print("Test Split Evaluation:")
        result = model.evaluate(test_datasets[i])
        result_list.append(result)
        model.save(save_path, save_format='tf')
    for result in result_list:
        loss_list.append(result[0])
        accuracy_list.append(result[1])
    print("\n\nAverage Loss: ", sum(loss_list)/len(loss_list))
    print("Average Accuracy: ", sum(accuracy_list)/len(accuracy_list))

# ResNet50 (ImageNet) Transfer Learning with no Fine Tuning

In [9]:
train(trainable_or_not=False, unfrozen_layers_count=0, epoch_count=10, save_directory='saved_models/no-finetune/',
      unlabeled_model_name='no-finetune', train_datasets=train_ds_list, test_datasets=test_ds_list)

Split 1:
Epoch 1/10
162/162 [==============================] - 16s 38ms/step - loss: 4.3942 - accuracy: 0.0389
Epoch 2/10
162/162 [==============================] - 9s 37ms/step - loss: 4.2364 - accuracy: 0.0491
Epoch 3/10
162/162 [==============================] - 9s 37ms/step - loss: 4.1897 - accuracy: 0.0544
Epoch 4/10
162/162 [==============================] - 9s 37ms/step - loss: 4.1728 - accuracy: 0.0479
Epoch 5/10
162/162 [==============================] - 9s 37ms/step - loss: 4.1375 - accuracy: 0.0583
Epoch 6/10
162/162 [==============================] - 9s 38ms/step - loss: 4.1095 - accuracy: 0.0587
Epoch 7/10
162/162 [==============================] - 9s 38ms/step - loss: 4.0796 - accuracy: 0.0647
Epoch 8/10
162/162 [==============================] - 9s 37ms/step - loss: 4.0855 - accuracy: 0.0533
Epoch 9/10
162/162 [==============================] - 10s 40ms/step - loss: 4.0632 - accuracy: 0.0584
Epoch 10/10
162/162 [==============================] - 9s 37ms/step - loss: 4.04

C:\Users\RGWLYD\miniconda3\envs\tfgpu\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Split 2:
Epoch 1/10
162/162 [==============================] - 12s 38ms/step - loss: 4.4036 - accuracy: 0.0435
Epoch 2/10
162/162 [==============================] - 10s 38ms/step - loss: 4.2276 - accuracy: 0.0438
Epoch 3/10
162/162 [==============================] - 10s 43ms/step - loss: 4.1995 - accuracy: 0.0542
Epoch 4/10
162/162 [==============================] - 10s 38ms/step - loss: 4.1559 - accuracy: 0.0514
Epoch 5/10
162/162 [==============================] - 10s 38ms/step - loss: 4.1021 - accuracy: 0.0638
Epoch 6/10
162/162 [==============================] - 10s 38ms/step - loss: 4.0834 - accuracy: 0.0562
Epoch 7/10
162/162 [==============================] - 10s 38ms/step - loss: 4.0779 - accuracy: 0.0634
Epoch 8/10
162/162 [==============================] - 10s 38ms/step - loss: 4.0467 - accuracy: 0.0582
Epoch 9/10
162/162 [==============================] - 10s 38ms/step - loss: 4.0330 - accuracy: 0.0634
Epoch 10/10
162/162 [==============================] - 10s 38ms/step - lo

# ResNet50 (ImageNet) Transfer Learning with Fine Tuning
Keep first 15 layeres of base model frozen from training and 160 unfrozen for training for optimal results.

In [10]:
train(trainable_or_not=True, unfrozen_layers_count=160, epoch_count=10, save_directory='saved_models/unfreeze160/',
      unlabeled_model_name='N-160', train_datasets=train_ds_list, test_datasets=test_ds_list)

Split 1:
Epoch 1/10
162/162 [==============================] - 27s 111ms/step - loss: 4.1121 - accuracy: 0.0648
Epoch 2/10
162/162 [==============================] - 21s 107ms/step - loss: 2.8588 - accuracy: 0.2021
Epoch 3/10
162/162 [==============================] - 21s 108ms/step - loss: 2.2951 - accuracy: 0.3157
Epoch 4/10
162/162 [==============================] - 21s 107ms/step - loss: 1.9425 - accuracy: 0.4189
Epoch 5/10
162/162 [==============================] - 21s 108ms/step - loss: 1.6240 - accuracy: 0.4984
Epoch 6/10
162/162 [==============================] - 21s 107ms/step - loss: 1.3146 - accuracy: 0.5889
Epoch 7/10
162/162 [==============================] - 21s 108ms/step - loss: 1.0503 - accuracy: 0.6767
Epoch 8/10
162/162 [==============================] - 21s 107ms/step - loss: 0.7909 - accuracy: 0.7561
Epoch 9/10
162/162 [==============================] - 21s 108ms/step - loss: 0.6694 - accuracy: 0.7957
Epoch 10/10
162/162 [==============================] - 21s 107ms

# Model Loading and Evaluating (No Training)

When using this section, restart the kernel and do not run the above cell.

In [ ]:
# model1 = tf.keras.models.load_model('saved_models/retrain-yes/model1-imgnet-yesretrain.h5')
# model2 = tf.keras.models.load_model('saved_models/retrain-yes/model2-imgnet-yesretrain.h5')
# model3 = tf.keras.models.load_model('saved_models/retrain-yes/model3-imgnet-yesretrain.h5')
# model4 = tf.keras.models.load_model('saved_models/retrain-yes/model4-imgnet-yesretrain.h5')
# model5 = tf.keras.models.load_model('saved_models/retrain-yes/model5-imgnet-yesretrain.h5')

# result_list = []
# loss_list = []
# accuracy_list = []

# print("Retrain N-0 layers (ALL)")
# print("Evaluating split 1:")
# result1 = model1.evaluate(ds_test1)
# result_list.append(result1)

# print("\nEvaluating split 2:")
# result2 = model2.evaluate(ds_test2)
# result_list.append(result2)

# print("\nEvaluating split 3:")
# result3 = model3.evaluate(ds_test3)
# result_list.append(result3)

# print("\nEvaluating split 4:")
# result4 = model4.evaluate(ds_test4)
# result_list.append(result4)

# print("\nEvaluating split 5:")
# result5 = model5.evaluate(ds_test5)
# result_list.append(result5)
# for result in result_list:
#     loss_list.append(result[0])
#     accuracy_list.append(result[1])

# print("\n\n\nAverage Loss: ", sum(loss_list)/len(loss_list))
# print("Average Accuracy: ", sum(accuracy_list)/len(accuracy_list))